## Using updated datasets in modeling worker Predictions

Please refer to this [documentation](https://datarobot-public-api-client.readthedocs-hosted.com/en/latest-release/reference/data/feature_discovery.html#make-predictions-with-a-trained-model) for additional details.

In [1]:
import datarobot as dr

In [10]:
dr.Client(token='',
          endpoint='https://app.datarobot.com/api/v2')

### Setup
Get the specific project and model 

In [3]:
project = dr.Project.get('66f4f57e56b6c62a3b2af7df')
project

Project(Lending+Club+Target.csv)

In [4]:
model = project.get_top_model()
model

/opt/anaconda3/envs/devenv311/lib/python3.11/site-packages/datarobot/models/project.py:2136: DataRobotDeprecationWarning: `get_models` has been marked for change in `3.4`, will be changed in `4.0`. In 4.0, the flag `use_new_models_retrieval` will be always `True`.
         Fewer attributes will be returned in the response, see the docstring for more details. 
  models = self.get_models()


Model('Light Gradient Boosted Trees Classifier with Early Stopping')

### Training Configuration
Get default configuration which is secondary datasets used in training for reference items like identifiers

In [37]:
secondary_dataset_configs = dr.SecondaryDatasetConfigurations.list(project.id)
secondary_dataset_configs

In [36]:
secondary_dataset_configs[0].to_dict()

{'id': '66f4f8399858adf6ef867e78',
 'project_id': '66f4f57e56b6c62a3b2af7df',
 'config': [{'feature_engineering_graph_id': '66f4f8039858adf6ef867e71',
   'secondary_datasets': [{'identifier': 'Lending+Club+Transac',
     'catalog_id': '66f4f5ed728bd2f567902bcc',
     'catalog_version_id': '66f4f5ed728bd2f567902bcd',
     'snapshot_policy': 'latest'}]},
  {'feature_engineering_graph_id': '66f4f8039858adf6ef867e72',
   'secondary_datasets': [{'identifier': 'Lending+Club+Profile',
     'catalog_id': '66f4f78da4c4c23bfa9029aa',
     'catalog_version_id': '66f4f78da4c4c23bfa9029ab',
     'snapshot_policy': 'latest'}]}],
 'name': 'Default Configuration',
 'secondary_datasets': [{'identifier': 'Lending+Club+Transac',
   'catalog_id': '66f4f5ed728bd2f567902bcc',
   'catalog_version_id': '66f4f5ed728bd2f567902bcd',
   'snapshot_policy': 'latest'},
  {'identifier': 'Lending+Club+Profile',
   'catalog_id': '66f4f78da4c4c23bfa9029aa',
   'catalog_version_id': '66f4f78da4c4c23bfa9029ab',
   'snapsh

### New Prediction time configuration
Create new secondary configuration with new datasets from AI catalog

In [12]:
id_sec_table_profile = '66f4ff3fa6701911f5902904'
version_id_sec_table_profile = '66f4ff3fa6701911f5902905'
id_sec_table_trasactions = '66f4ff82a6701911f590293c'
version_id_sec_table_trasactions = '66f4ff82a6701911f590293d'

In [14]:
profile_secondary_dataset = dr.SecondaryDataset(
    identifier='Lending+Club+Profile',
    catalog_id=id_sec_table_profile,
    catalog_version_id=version_id_sec_table_profile,
    snapshot_policy='latest'
)

transaction_secondary_dataset = dr.SecondaryDataset(
    identifier='Lending+Club+Transac',
    catalog_id=id_sec_table_trasactions,
    catalog_version_id=version_id_sec_table_trasactions,
    snapshot_policy='latest'
)

secondary_datasets = [profile_secondary_dataset, transaction_secondary_dataset]

new_secondary_dataset_config = dr.SecondaryDatasetConfigurations.create(
    project_id=project.id,
    name='New Configuration for Prediction',
    secondary_datasets=secondary_datasets
)

Verifying that the new configuration is linked to project

In [38]:
secondary_dataset_configs[1].to_dict()

{'id': '66f50299ff9ec635a4fceca4',
 'project_id': '66f4f57e56b6c62a3b2af7df',
 'config': [{'feature_engineering_graph_id': '66f4f8039858adf6ef867e72',
   'secondary_datasets': [{'identifier': 'Lending+Club+Profile',
     'catalog_id': '66f4ff3fa6701911f5902904',
     'catalog_version_id': '66f4ff3fa6701911f5902905',
     'snapshot_policy': 'latest'}]},
  {'feature_engineering_graph_id': '66f4f8039858adf6ef867e71',
   'secondary_datasets': [{'identifier': 'Lending+Club+Transac',
     'catalog_id': '66f4ff82a6701911f590293c',
     'catalog_version_id': '66f4ff82a6701911f590293d',
     'snapshot_policy': 'latest'}]}],
 'name': 'New Configuration for Prediction',
 'secondary_datasets': [{'identifier': 'Lending+Club+Profile',
   'catalog_id': '66f4ff3fa6701911f5902904',
   'catalog_version_id': '66f4ff3fa6701911f5902905',
   'snapshot_policy': 'latest'},
  {'identifier': 'Lending+Club+Transac',
   'catalog_id': '66f4ff82a6701911f590293c',
   'catalog_version_id': '66f4ff82a6701911f590293d',

### Predictions with new Configuration
make predictions on model blueprints using newly uploaded configuration

In [15]:
dataset_from_path = project.upload_dataset(
    './Lending+Club+Target.csv',
    secondary_datasets_config_id=new_secondary_dataset_config.id
)

predict_job_1 = model.request_predictions(dataset_from_path.id)

In [25]:
predict_job_1

PredictJob(Model('66f4f8afef23edf9988d5e0f'), status=COMPLETED)

In [26]:
predict_job_1.get_result_when_complete()

,row_id,prediction,positive_probability,prediction_threshold,class_No,class_Yes
0,0,No,0.027765,0.5,0.972235,0.027765
1,1,No,0.070605,0.5,0.929395,0.070605
2,2,No,0.013161,0.5,0.986839,0.013161
3,3,No,0.038213,0.5,0.961787,0.038213
4,4,No,0.021752,0.5,0.978248,0.021752
...,...,...,...,...,...,...
9995,9995,No,0.015054,0.5,0.984946,0.015054
9996,9996,No,0.050740,0.5,0.949260,0.050740
9997,9997,No,0.040809,0.5,0.959191,0.040809
9998,9998,No,0.037467,0.5,0.962533,0.037467


In [22]:
predict_job_1.get_predictions(project_id=project.id, predict_job_id=predict_job_1.id)

,row_id,prediction,positive_probability,prediction_threshold,class_No,class_Yes
0,0,No,0.027765,0.5,0.972235,0.027765
1,1,No,0.070605,0.5,0.929395,0.070605
2,2,No,0.013161,0.5,0.986839,0.013161
3,3,No,0.038213,0.5,0.961787,0.038213
4,4,No,0.021752,0.5,0.978248,0.021752
...,...,...,...,...,...,...
9995,9995,No,0.015054,0.5,0.984946,0.015054
9996,9996,No,0.050740,0.5,0.949260,0.050740
9997,9997,No,0.040809,0.5,0.959191,0.040809
9998,9998,No,0.037467,0.5,0.962533,0.037467
